In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import xesmf as xe

import os
import sys
from pathlib import Path

HOME = Path(os.getcwd()).parents[0]

sys.path.insert(1, str(HOME))

In [32]:
from dsrnngan.data import VAR_LOOKUP_ERA5, load_hdf5_file, filter_by_lat_lon, get_ifs_filepath, get_era5_filepath_prefix, get_era5_path, load_imerg_raw, get_era5_stats, filter_by_lat_lon, load_imerg_raw

In [33]:
# read some sample data in 
# - Check units
# - check orientation of lat and long
# - Check times

# Definitions are given here: https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation

### Notes

tp appears to be the total rainfall on a given day, in metres. NOTE: this is on a 0.25 degree grid (the rest are on a 1 degree grid)
https://apps.ecmwf.int/codes/grib/param-db?id=228

shum: I can't find in any of the definition urls, but it does have q as a variable which aligns
https://apps.ecmwf.int/codes/grib/param-db?id=133
kg kg-1 (mass of water vapour per kg of moist air)

ta: Near surface air temperature . Presumably Kelvin?
https://clipc-services.ceda.ac.uk/dreq/u/bab9237c-e5dd-11e5-8482-ac72891c3257.html

u: u component of wind (eastward)
https://apps.ecmwf.int/codes/grib/param-db?id=131

v: v component of wind (northward)
https://apps.ecmwf.int/codes/grib/param-db?id=132

I think time_bnds is time bounds, presumably time bounds of data gathering? Do we need to check that some days don't have timny time windows?

Rough corners of the box we are considering (1000km x 1000km)
SE: -2.928111255616329, 43.25853873313056
NE: 6.307770957731585, 43.25853873313056
SW: -2.928111255616329, 33.80008363706967
NW: 6.307770957731585, 33.80008363706967



In [7]:
variable = 'tp'
year = 2020
month = 11
latitude_vals = list(range(-3, 8))
longitude_vals = list(range(33, 44))
log_precip = False

In [8]:
year = 2020
month = 11
day = 1
data_folder = HOME / 'system_tests' / 'data'
era5_path = str(data_folder / 'ERA5')
ds = xr.load_dataset(get_era5_path(variable, year=year, month=month, era_data_dir=era5_path))

FileNotFoundError: [Errno 2] No such file or directory: b'/user/home/uz22147/repos/downscaling-cgan/system_tests/data/ERA5/total_precipitation/ERA5_total_precipitation_day_202011.nc'

In [5]:
# Get small subset of ERA5 data

for v in VAR_LOOKUP_ERA5:
    input_year = 2018
    for input_month in range(1,13):

        latitude_vals=[-2, -1, 0 , 1, 2]
        longitude_vals=[32, 33, 34, 35]
        ds = xr.load_dataset(get_era5_path(v, year=input_year, month=input_month))

        lon_var_name = 'longitude' if v == 'tp' else 'lon'
        lat_var_name = 'latitude' if v == 'tp' else 'lat'

        ds = filter_by_lat_lon(ds, lon_range=longitude_vals, 
                                   lat_range=latitude_vals,
                                   lon_var_name=lon_var_name,
                                   lat_var_name=lat_var_name)

        fp = get_era5_path(variable=v, year=input_year, month=input_month, era_data_dir=str(HOME / 'system_tests/data/ERA5'))
        ds.to_netcdf(fp)


In [4]:
# Get small sample of imerg data
from datetime import datetime
from glob import glob
latitude_vals=[-2, -1, 0 , 1, 2]
longitude_vals=[32, 33, 34, 35]
imerg_data_dir = '/bp1/geog-tropical/data/Obs/IMERG/half_hourly/final/'
glob_str = os.path.join(imerg_data_dir, '3B-HHR.MS.MRG.3IMERG.2018123*-S18*')
glob_str_2 = os.path.join(imerg_data_dir, '3B-HHR.MS.MRG.3IMERG.20170704*-S18*')
glob_str_3 = os.path.join(imerg_data_dir, '3B-HHR.MS.MRG.3IMERG.2018123*-S17*')
glob_str_4 = os.path.join(imerg_data_dir, '3B-HHR.MS.MRG.3IMERG.20170704*-S17*')

fps = set(glob(glob_str) + glob(glob_str_2) + glob(glob_str_3) + glob(glob_str_4))

for fp in fps:
    output_imerg_path = os.path.join('../system_tests/data/IMERG/half_hourly/final', fp.split('/')[-1].replace('HDF5', 'nc'))
    print(output_imerg_path)
    ds = load_hdf5_file(fp)
    small_ds = filter_by_lat_lon(ds, lon_range=longitude_vals, 
                                   lat_range=latitude_vals,
                                   lon_var_name='lon',
                                   lat_var_name='lat')
    
    small_ds.to_netcdf(output_imerg_path)

../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20181230-S183000-E185959.1110.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20181230-S173000-E175959.1050.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20170704-S183000-E185959.1110.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20181230-S170000-E172959.1020.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20170704-S173000-E175959.1050.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20170704-S170000-E172959.1020.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20181231-S170000-E172959.1020.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20181230-S180000-E182959.1080.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMERG.20170704-S180000-E182959.1080.V06B.nc
../system_tests/data/IMERG/half_hourly/final/3B-HHR.MS.MRG.3IMER

In [27]:
year = 2018
month = 12
day = 30
hour = 18
latitude_vals = [0, 0.1, 0.2, 0.3, 0.4]
longitude_vals = [33, 34]

ds = load_imerg_raw(year=year, month=month, day=day, hour=hour)

In [28]:
# original_lat_vals = ds.lat.values
# new_lat_vals = ds.lat.values + 1e-6
# ds.reindex(lat=list(set(new_lat_vals)), lon=list(set(ds.lon.values)), latv=list(set(ds.latv.values)), lonv=list(set(ds.lonv.values)))
ds = ds.sel(lat=latitude_vals, method='backfill')
print(ds.lat.values)

[0.05 0.15 0.25 0.35 0.45]


In [18]:
get_ifs_filepath('tp', loaddate=datetime(2017, 7, 4), loadtime='*')

'/bp1/geog-tropical/users/uz22147/east_africa_data/IFS/tp/tp_HRES_1h_EAfrica_2017-07-04_*h.nc'

In [34]:

# Get a sample of IFS data
import glob
from calendar import monthrange

from dsrnngan.data import all_ifs_fields, get_ifs_filepath
latitude_vals=[-2, -1, 0 , 1, 2]
longitude_vals=[32, 33, 34, 35]

year = 2017
month = 7

for day in [4, 5]:
    for var in all_ifs_fields:
        glob_str = get_ifs_filepath(var, loaddate=datetime(year, month ,day), loadtime='*')
        fps = glob.glob(glob_str)
        for fp in fps:
            suffix = fp.split('/')[-1]
            loadtime = int(suffix.split('.')[0].split('_')[-1].replace('h', ''))
            ds = xr.open_dataset(fp)
            small_ds = filter_by_lat_lon(ds, lon_range=longitude_vals, lat_range=latitude_vals)
            output_fp = get_ifs_filepath(var, loaddate=datetime(year, month, day), loadtime=loadtime, fcst_dir='../system_tests/data/IFS/')
            small_ds.to_netcdf(output_fp)


                # vars = [fp.split('/')[-1].split('_')[0] for fp in fps]
                # for n, var in enumerate(vars):

                #     fp = [fp for fp in fps if fp.split('/')[-1].startswith(var)][0]
                #     suffix = fp.split('/')[-1]
                #     ds = xr.open_dataset(fp)
                #     var_name = list(ds.data_vars)[0]
                    
                #     small_ds = filter_by_lat_lon(ds, lon_range=longitude_vals, lat_range=latitude_vals)
                #     sample = small_ds.sel(time=slice(f"{year}-07-05T04", "2017-07-05T05"))

                #     sample.to_netcdf(f'../system_tests/data/IFS/{suffix}')


In [10]:
import glob
year = 2018
month = 12
day = 31

glob.glob(f'/bp1/geog-tropical/users/uz22147/IFS_east_africa/tp/*_HRES_1h_EAfrica_{year}-{month:02d}-{day:02d}*.nc')

[]

In [4]:
ds = xr.load_dataset('/user/home/uz22147/repos/downscaling-cgan/system_tests/data/IFS/r/r200/r200_HRES_1h_EAfrica_2017-07-04_12h.nc')

In [13]:
len([item for item in ds['r'].values.flatten() if item < 0]) / len( ds['r'].values.flatten())

0.0

In [19]:
# get a sample of nimrod data
path_2018 = '/user/home/uz22147/repos/downscaling-cgan/system_tests/data/NIMROD/2018/metoffice-c-band-rain-radar_uk_20181231.nc'
path_2019 = '/user/home/uz22147/repos/downscaling-cgan/system_tests/data/NIMROD/2019/metoffice-c-band-rain-radar_uk_20191231.nc'

ds_2018 = xr.open_dataset(path_2018)
# .sel(latitude=[-1, 0,1, 2], longitude=[32, 33, 34, 35], method='nearest')
ds_2019 = xr.open_dataset(path_2019)
# .sel(latitude=[-1, 0,1, 2], longitude=[32, 33, 34, 35], method='nearest')

# ds_2018.to_netcdf('/user/home/uz22147/repos/downscaling-cgan/system_tests/data/NIMROD/2018/metoffice-c-band-rain-radar_uk_20181231.nc')
# ds_2019.to_netcdf('/user/home/uz22147/repos/downscaling-cgan/system_tests/data/NIMROD/2019/metoffice-c-band-rain-radar_uk_20191231.nc')

In [9]:
#sample of constants
latitude_vals=[-2, -1, 0 , 1, 2]
longitude_vals=[32, 33, 34, 35]

h_ds = xr.open_dataset('/bp1/geog-tropical/users/uz22147/east_africa_data/constants/h_HRES_EAfrica.nc')
lsm_ds = xr.open_dataset('/bp1/geog-tropical/users/uz22147/east_africa_data/constants/lsm_HRES_EAfrica.nc')

h_ds = filter_by_lat_lon(h_ds, lon_range=longitude_vals, lat_range=latitude_vals)
lsm_ds = filter_by_lat_lon(lsm_ds, lon_range=longitude_vals, lat_range=latitude_vals)

h_ds.to_netcdf(f'../system_tests/data/constants/h_HRES_EAfrica.nc')
lsm_ds.to_netcdf(f'../system_tests/data/constants/lsm_HRES_EAfrica.nc')

In [ ]:
test_folder = "../system_tests/data/tfrecords/IFS_nimrod/"

fle_hdles = []
year = 2018
hour = 9



In [11]:
# Get a sample of tfrecords
import tensorflow as tf

for fh in range(4):
    flename = os.path.join(f"../system_tests/data/tfrecords/IFS_nimrod/", 'samples', f"2018_9.{fh}.tfrecords")
    file_handle = tf.io.TFRecordWriter(flename)
    
    raw_dataset = tf.data.TFRecordDataset(f"../system_tests/data/tfrecords/IFS_nimrod/2018_9.{fh}.tfrecords")
    
    for raw_record in raw_dataset.take(100):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        example_to_string = example.SerializeToString()
        file_handle.write(example_to_string)

    file_handle.close()
    
    

In [14]:
# validation set
flename = os.path.join(f"../system_tests/data/tfrecords/IFS_nimrod/", 'samples', f"validation2019.tfrecords")

file_handle = tf.io.TFRecordWriter(flename)
raw_dataset = tf.data.TFRecordDataset(f"../system_tests/data/tfrecords/IFS_nimrod/validation2019.tfrecords")

for raw_record in raw_dataset.take(100):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    
    example_to_string = example.SerializeToString()
    file_handle.write(example_to_string)

file_handle.close()

In [28]:
# Same for ERA5 / iMERG

import tensorflow as tf

for fh in range(4):
        
    raw_dataset = tf.data.TFRecordDataset(f"/user/work/uz22147/tfrecords/era5_imerg/2018_9.{fh}.tfrecords")
    
    flename = os.path.join(f"../system_tests/data/tfrecords/ERA5_iMERG/", f"2018_9.{fh}.tfrecords")
    file_handle = tf.io.TFRecordWriter(flename)

    m = 0
    for raw_record in raw_dataset.take(1000):
        print(m)
        m+=1
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        example_to_string = example.SerializeToString()
        file_handle.write(example_to_string)

    file_handle.close()


0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5


In [7]:
from glob import glob

fl = glob(f"/user/home/uz22147/repos/downscaling-cgan/data/tfrecords/2018_9.0.tfrecords")

ds = tf.data.TFRecordDataset(fl)
insize=(20, 20, 9)
consize=(200, 200, 2)
outsize=(200, 200, 1)
    # Create a description of the features
feature_description = {
    'generator_input': tf.io.FixedLenFeature(insize, tf.float32),
    'constants': tf.io.FixedLenFeature(consize, tf.float32),
    'generator_output': tf.io.FixedLenFeature(outsize, tf.float32),
}
for raw_record in ds.take(100):
    example = tf.io.parse_example(raw_record, feature_description)
